<a href="https://colab.research.google.com/github/Dxmoore/Forest-Cover-Classification-with-Deep-Learning/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


###Loading Data###

#import data
data = pd.read_csv('cover_data.csv')

#view data columns and types
data.info()

# Check for NaN values in the 'class' column-- found one row with nan value 
nan_values = data['class'].isna().sum()

# Print the number of NaN values
print("Number of NaN values in 'class' column:", nan_values)

# Drop rows with NaN values in the 'class' column
data = data.dropna(subset=['class'])

#view distribution of the target column 'death event'
print('Classes and number of values in the dataset', Counter(data['class'])) 

#create variable for label and features column
y = data['class']
x = data.drop(['class'], axis = 1)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251506 entries, 0 to 251505
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Elevation                           251506 non-null  int64  
 1   Aspect                              251506 non-null  int64  
 2   Slope                               251506 non-null  int64  
 3   Horizontal_Distance_To_Hydrology    251506 non-null  int64  
 4   Vertical_Distance_To_Hydrology      251506 non-null  int64  
 5   Horizontal_Distance_To_Roadways     251506 non-null  int64  
 6   Hillshade_9am                       251506 non-null  int64  
 7   Hillshade_Noon                      251506 non-null  int64  
 8   Hillshade_3pm                       251506 non-null  int64  
 9   Horizontal_Distance_To_Fire_Points  251506 non-null  int64  
 10  Wilderness_Area1                    251506 non-null  int64  
 11  Wilderness_Area2          

The dataset contains a great imbalance in classifications, leaning heavily towards classes 1 and 2 (Spruce/Fir
Lodgepole Pine). 

In [20]:
###Data Preprocessing###


#split the data
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 40)


#scale the numeric training and test feature values
numerical_columns = data.select_dtypes(include=['int64']).columns
ct = ColumnTransformer([("only numeric", StandardScaler(), x.columns)], remainder='passthrough')#used x.columns since all data is in int64
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)







In [ ]:
###Create Model###

# Determine the number of classes
num_classes = len(np.unique(y))

# Subtract 1 from class labels -> class lables must be zero indexed to use with sparse categorical crossentrophy
Y_train -= 1


# Define the early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)

#create model instance with input, hidden and output layers 
model = Sequential()
model.add(InputLayer(input_shape=(X_train_scaled.shape[1],))) #input
model.add(Dense(12, activation='relu')) #hidden
model.add(Dense(num_classes, activation='softmax')) #output
# Use sparse categorical cross-entropy as the loss function
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #compile model
model.fit(X_train_scaled, Y_train, epochs=100, batch_size=32, callbacks = [early_stopping]) # fit model
loss, acc = model.evaluate(X_test_scaled, Y_test)





Epoch 1/100
6288/6288 [==============================] - 21s 3ms/step - loss: 0.6152 - accuracy: 0.7517
Epoch 2/100
6288/6288 [==============================] - 23s 4ms/step - loss: 0.5146 - accuracy: 0.7796
Epoch 3/100
6288/6288 [==============================] - 20s 3ms/step - loss: 0.5003 - accuracy: 0.7849
Epoch 4/100
6288/6288 [==============================] - 18s 3ms/step - loss: 0.4922 - accuracy: 0.7899
Epoch 5/100
6288/6288 [==============================] - 18s 3ms/step - loss: 0.4871 - accuracy: 0.7932
Epoch 6/100
6288/6288 [==============================] - 18s 3ms/step - loss: 0.4830 - accuracy: 0.7948
Epoch 7/100
6288/6288 [==============================] - 18s 3ms/step - loss: 0.4804 - accuracy: 0.7964
Epoch 8/100
6288/6288 [==============================] - 22s 3ms/step - loss: 0.4776 - accuracy: 0.7983
Epoch 9/100
6288/6288 [==============================] - 17s 3ms/step - loss: 0.4752 - accuracy: 0.7990
Epoch 10/100
6288/6288 [==============================] - 18s 3m

In [21]:
### Generate a classification report ###
y_estimate = model.predict(X_test_scaled, verbose=0)
y_estimate = np.argmax(y_estimate, axis=1)
print(classification_report(Y_test, y_estimate))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00   14477.0
         2.0       0.00      0.00      0.00   31276.0
         3.0       0.00      0.00      0.00    1252.0
         4.0       0.00      0.00      0.00     433.0
         5.0       0.00      0.00      0.00    1130.0
         6.0       0.00      0.00      0.00     978.0
         7.0       0.00      0.00      0.00     755.0

    accuracy                           0.00   50301.0
   macro avg       0.00      0.00      0.00   50301.0
weighted avg       0.00      0.00      0.00   50301.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.